In [1]:
#훈련 중, 훈련 후 모델 저장 가능
#모델중지 지점부터 다시 훈련 가능
#모델 공유(모델 코드, weight, parameter) 가능

In [2]:
#설치와 임포트 ; 필요한 라이브러리 설치 및 텐서플로 임포트

In [3]:
pip install -q pyyaml h5py

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.0.0-rc1


In [2]:
#예제 데이터셋 받기 ; MNIST 데이터셋, 모델 실행 속도 빠르게 하기 위해 처음 1,000개만 사용
(train_images, train_labels), (test_images,test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

#신경망 모델에 주입하기 전에 이 값의 범위를 0~1사이로 조정
#훈련세트와 테스트 세트를 동일한 방식으로 전처리하는 것이 중요하다!!
#reshape(-1,28*28)은 열은 28*28 size, 행은 남은 차원
train_images = train_images[:1000].reshape(-1,28*28) / 255.0
test_images = test_images[:1000].reshape(-1,28*28) / 255.0

In [3]:
#모델 정의
#간단한 Sequential 모델 정의
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation = 'relu', input_shape = (784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    
    model.compile(optimizer = 'adam',
                  #tf에서 loss설정방법은 3가지가 있다.
                  #1번째는 lossfunction의 이름을 문자열로 전달
                  #2번째는 tf.keras.losses의 손실 함수 사용
                  #3번째는 tf.keras.losses의 손실함수 클래스 사용
                  #logit? 모델 출력값이 문제에 맞게 normalize가 되었느냐
                  #즉, 이미지 분류 문제에서 출력값이 확률이면 logit = False
                  #출력값이 sigmoid, linear라면 logit = true
                 loss = tf.losses.SparseCategoricalCrossentropy(from_logits = True),
                 metrics = ['accuracy'])
    return model

#모델 객체 만들기
model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
#훈련하는 동안 체크포인트 저장하기 ; 다시 훈련하지 않고 모델 재사용 또는 이어서 훈련 가능
#tf.keras.callbacks.ModelCheckpoint

In [5]:
#체크포인트 콜백 사용하기
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                save_weights_only = True,
                                                verbose = 1)
#새로운 콜백으로 모델 훈련하기
model.fit(train_images,
         train_labels,
         epochs = 10,
         validation_data = (test_images, test_labels),
         callbacks = [cp_callback]) #콜백을 훈련에 전달

#텐서플로 체크포인트를 만들고 에포크 종료될때마다 업데이트

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 576/1000 [================>.............] - ETA: 0s - loss: 1.4890 - accuracy: 0.5764 
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 584us/sample - loss: 1.1555 - accuracy: 0.6790 - val_loss: 0.7104 - val_accuracy: 0.7870
Epoch 2/10
 576/1000 [================>.............] - ETA: 0s - loss: 0.4519 - accuracy: 0.8681
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 189us/sample - loss: 0.4137 - accuracy: 0.8820 - val_loss: 0.5229 - val_accuracy: 0.8350
Epoch 3/10
 576/1000 [================>.............] - ETA: 0s - loss: 0.2856 - accuracy: 0.9236
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 187us/sample - loss: 0.2775 - accuracy: 0.9290 - val_loss: 0.4493 - val_accuracy: 0.8550
Epoch 4/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.2130 - accuracy: 0.9448
Epoc

In [6]:
ls {checkpoint_dir}

 C 드라이브의 볼륨: OS
 볼륨 일련 번호: F423-2466

 C:\Users\82104\training_1 디렉터리

2021-02-18  오후 11:30    <DIR>          .
2021-02-18  오후 11:30    <DIR>          ..
2021-02-18  오후 11:30                71 checkpoint
2021-02-18  오후 11:30         4,886,688 cp.ckpt.data-00000-of-00001
2021-02-18  오후 11:30             1,222 cp.ckpt.index
               3개 파일           4,887,981 바이트
               2개 디렉터리  254,184,656,896 바이트 남음


In [7]:
#훈련하지 않은 새로운 모델 생성
#가중치만 복원할 때에는 원본 모델과 동일한 구조로 모델 생성해야함!!
#여기서는 동일한 구조로 모델을 만들어서 다른 객체지만 가중치 공유 가능

#기본 모델 객체 생성 & test set에서 평가
model = create_model()

#모델 평가
loss, acc = model.evaluate(test_images,test_labels, verbose = 2)
print("훈련되지 않은 모델의 정확도 : {:0.2f}%".format(100*acc))

#훈련되지 않은 모델의 성능은 무작위로 선택하는 정도 10% 수준

1000/1 - 0s - loss: 2.2703 - accuracy: 0.1170
훈련되지 않은 모델의 정확도 : 11.70%


In [8]:
#체크포인트에서 가중치 로드하고 다시 평가하기
#가중치 로드
model.load_weights(checkpoint_path)

#model Re-evaluation
loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print("복원된 모델의 정확도 : {:0.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.6057 - accuracy: 0.8660
복원된 모델의 정확도 : 86.60%


In [9]:
#체크포인트 콜백 매개변수 ; 체크포인트 이름 고유하게 만들기, 체크포인트 주기 설정
#파일이름에 에포크 번호를 포함시킨다('str.format'포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#다섯 번째 에포크마다 가중치를 저장하기위한 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(
               filepath = checkpoint_path,
               verbose = 1,
               save_weights_only = True,
               period = 5)

#새로운 모델 객체 생성
model = create_model()

#'checkpoint_path'포맷을 사용하는 가중치를 저장
model.save_weights(checkpoint_path.format(epoch=0))

#새로운 콜백을 사용하여 모델을 훈련
model.fit(train_images,
         train_labels,
         epochs = 50,
         callbacks = [cp_callback],
         validation_data = (test_images,test_labels),
         verbose = 0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [10]:
ls {checkpoint_dir}

 C 드라이브의 볼륨: OS
 볼륨 일련 번호: F423-2466

 C:\Users\82104\training_2 디렉터리

2021-02-18  오후 11:31    <DIR>          .
2021-02-18  오후 11:31    <DIR>          ..
2021-02-18  오후 11:31                81 checkpoint
2021-02-18  오후 11:31         1,628,741 cp-0000.ckpt.data-00000-of-00001
2021-02-18  오후 11:31               402 cp-0000.ckpt.index
2021-02-18  오후 11:31         4,886,700 cp-0005.ckpt.data-00000-of-00001
2021-02-18  오후 11:31             1,222 cp-0005.ckpt.index
2021-02-18  오후 11:31         4,886,700 cp-0010.ckpt.data-00000-of-00001
2021-02-18  오후 11:31             1,222 cp-0010.ckpt.index
2021-02-18  오후 11:31         4,886,700 cp-0015.ckpt.data-00000-of-00001
2021-02-18  오후 11:31             1,222 cp-0015.ckpt.index
2021-02-18  오후 11:31         4,886,700 cp-0020.ckpt.data-00000-of-00001
2021-02-18  오후 11:31             1,222 cp-0020.ckpt.index
2021-02-18  오후 11:31         4,886,700 cp-0025.ckpt.data-00000-of-00001
2021-02-18  오후 11:31             1,222 cp-0025.ckpt.index
2021-02-18  오후 1

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [12]:
#모델 초기화 후 최근 체크포인트 로드하여 테스트
#새로운 모델 객체 만들기
model = create_model()

#이전에 저장한 가중치를 로드
model.load_weights(latest)

#모델 재평가
loss,acc = model.evaluate(test_images,test_labels,verbose = 2)
print("복원된 모델의 정확도: {:0.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.5304 - accuracy: 0.8800
복원된 모델의 정확도: 88.00%


In [ ]:
#수동으로 가중치 저장하기
#가중치 저장하기
model.save_weights('./checkpoint/my_checkpoint')
#새로운 모델 객체 생성
model = create_model()
#가중치 복원
model.load_weights('./checkoint/my_checkpoint')
#모델 평가
loss,acc = model.evaluate(test_images,test_labels,verbose = 2)
print("복원된 모델의 정확도: {:0.2f}%".format(100*acc))

In [ ]:
#전체 모델 저장하기
#model.save() ; 모델 구조, weight, 훈련 설정을 하나의 폴더나 파일로 저장
#원본 파이썬 코드 없어도 되고 정확히 중지된 시점에서 훈련 재시작 가능(optimizer상태 복원)
#2개의 포맷(SavedModel, HDF5)으로 모델 저장 가능

#SavedModel 포맷 ; 모델을 직렬화하는 다른 방법
#새로운 모델 객체 만들고 훈련
model = create_model()
model.fit(train_images,train_labels,epochs = 5)

#SavedModel로 전체 모델 저장
!mkdir -p saved_model
model.save('saved_model/my_model')

In [ ]:
#지정된 모델로부터 새로운 케라스 모델 로드

new_model = tf.keras.models.load_model('saved_model/my_model')

#모델 구조 확인
new_model.summary()

#커널 계속 꺼짐 현상 발생

In [ ]:
#HDF5파일로 저장하기
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5')

In [ ]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = tf.keras.models.load_model('my_model.h5')

# 모델 구조를 출력합니다
new_model.summary()

In [ ]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

In [ ]:
#정리
#케라스는 모델 구조를 파악하여 정리(weight 값, 모델구조, 모델 훈련 설정;컴파일시 전달한 값, 옵티마이저)
